In [1]:
from requests import get
import pandas as pd
import json
import numpy as np 

In [2]:
# første sted...TV2! Fordi de gør det nemt og tidlig succes giver motivation til de mere besværlige steder.
# Til kommunalvalg, har de en json fil per "greaterConstituency".
# Greater constiuencies er gemt i en fil på forsiden...
# ...og nu har de åbenbart delt svarerne ind i en json per valgdistrikt
storkredse = json.load(open("./TV2/greaterConstituencies.json"))
småkredse = {101: 9000100, 147: 9000100, 151: 9000200, 153: 9000200, 155: 9000100, 157: 9000200, 159: 9000200, 161: 9000200, 163: 9000200, 165: 9000200, 167: 9000200, 169: 9000200, 173: 9000200, 175: 9000200, 183: 9000200, 185: 9000100, 187: 9000200, 190: 9000300, 201: 9000300, 210: 9000300, 217: 9000300, 219: 9000300, 223: 9000300, 230: 9000300, 240: 9000300, 250: 9000300, 253: 9000500, 259: 9000500, 260: 9000300, 265: 9000500, 269: 9000500, 270: 9000300, 306: 9000500, 316: 9000500, 320: 9000500, 326: 9000500, 329: 9000500, 330: 9000500, 336: 9000500, 340: 9000500, 350: 9000500, 360: 9000500, 370: 9000500, 376: 9000500, 390: 9000500, 400: 9000400, 410: 9000600, 420: 9000600, 430: 9000600, 440: 9000600, 450: 9000600, 461: 9000600, 479: 9000600, 480: 9000600, 482: 9000600, 492: 9000600, 510: 9000700, 530: 9000700, 540: 9000700, 550: 9000700, 561: 9000700, 563: 9000700, 573: 9000700, 575: 9000700, 580: 9000700, 607: 9000700, 615: 9000800, 621: 9000700, 630: 9000700, 657: 9000900, 661: 9000900, 665: 9000900, 671: 9000900, 706: 9000800, 707: 9000800, 710: 9000800, 727: 9000800, 730: 9000800, 740: 9000900, 741: 9000800, 746: 9000800, 751: 9000800, 756: 9000900, 760: 9000900, 766: 9000800, 773: 9001000, 779: 9000900, 787: 9001000, 791: 9000900, 810: 9001000, 813: 9001000, 820: 9001000, 825: 9001000, 840: 9001000, 846: 9001000, 849: 9001000, 851: 9001000, 860: 9001000}

In [3]:
# Kør kun første gang
#svar = {}
#for kreds in småkredse.keys():
#    print(kreds)
#    res = get(f"https://election-api.services.tv2.dk/kv/KV2025/candidatetest/answers/{kreds}")
#    svar[kreds] = res.json()
##gem svarerne
#json.dump(svar, open('./TV2/alle_svar.json', 'w'))

In [4]:
tv2_spg = [f'tv2-kv25-danmark-{x+1}' for x in range(20)]

In [5]:
# smæk det hele pænt sammen
svar = json.load(open('./TV2/alle_svar.json'))
tv2_svar = {}
tv2_kandidater = {}
for kreds_id, kandidater in svar.items():
    for kandidat in kandidater:
        tv2_kandidater[kandidat['id']] = {'navn': kandidat['name'], 'kreds': kandidat['areaName'], 'job': kandidat['occupation'], 'parti': kandidat['partyName'], 'alder': kandidat['age']}
        tv2_svar[kandidat['id']] = {x: kandidat['answers'][x]['answer'] for x in tv2_spg if x in kandidat['answers']}
tv2_kandidater = pd.DataFrame(tv2_kandidater).T
tv2_svar = pd.DataFrame(tv2_svar).T

In [6]:
# norm mellem 0 og 1
ma = tv2_svar.max()
mi = tv2_svar.min()
tv2_svar = (tv2_svar-mi)/(ma-mi)

In [7]:
q = tv2_svar.copy()
q['parti'] = [tv2_kandidater.loc[x, 'parti'] for x in tv2_svar.index]
q['navn'] = [tv2_kandidater.loc[x, 'navn'] for x in tv2_svar.index]
q['job'] = [tv2_kandidater.loc[x, 'job'] for x in tv2_svar.index]
q['alder'] = [tv2_kandidater.loc[x, 'alder'] for x in tv2_svar.index]
q['kreds'] = [tv2_kandidater.loc[x, 'kreds'] for x in tv2_svar.index]
#q.parti.replace("Konservative", "Det Konservative Folkeparti", inplace=True)

In [8]:
# dejligt nemt. Tak TV2s programmører!

In [9]:
# DR's data er...mere gemt...men jeg fandt en acceptabel måde at hive det ud på via scrapy
# Men selvfølgelig har de ændret på det i år...og nu er det hele gemt i et script tag i html filen.
# kør kommando i TakeTheDR mappen:
# scrapy crawl DR -O DR_data.json -L WARNING

alt_dr = json.load(open('./TakeTheDR/DR_data.json'))

In [10]:
#lad os lige kigge på data
alt_dr[0].keys()

dict_keys(['candidate', 'candidateAnswers', 'questions'])

In [11]:
alt_dr[0]['candidate'].keys()

dict_keys(['ID', 'Firstname', 'LastName', 'Picture', 'CroppedPicture', 'Photographer', 'Gender', 'Birthdate', 'UrlKey', 'PersonUrlKey', 'CurrentParty', 'CurrentPartyCode', 'City', 'Zipcode', 'Facebook', 'Homepage', 'Instagram', 'Twitter', 'Profession', 'Education', 'LeisureInterest', 'PoliticsInterest', 'LinedUpForParty', 'LinedUpForPartyCode', 'ElectionHistory', 'Keytopics', 'LineUps'])

In [12]:
alt_dr[0]['candidate']['LineUps']
gt = []
alu = []
for kan in alt_dr:
    alu.append(len(kan['candidate']['LineUps']))
    for lu in kan['candidate']['LineUps']:
        gt.append(lu['groupType'])

print(pd.Series(gt).value_counts())
print(pd.Series(alu).value_counts())
# hmm ok vi dropper regionsvalget 


Municipality    8195
Region          1023
Name: count, dtype: int64
1    9218
Name: count, dtype: int64


In [13]:
drdata = [] # tømmerne den lige for at undgå at skulle genskrive koden neden under
for kan in alt_dr:
    if kan['candidate']['LineUps'][0]['groupType'] == 'Municipality':
        drdata.append(kan)

In [14]:
drdata[0]['candidateAnswers'][0].keys()

dict_keys(['QuestionID', 'Answer', 'AnswerList', 'Info', 'IsImportant', 'ID_CandidateQuestionType', 'IsTopicQuestionType'])

In [15]:
drdata[0]['questions'][0].keys()

dict_keys(['Id', 'Title', 'Question', 'Info', 'ArgumentFor', 'ArgumentAgainst', 'OptionalTitle', 'OptionalQuestion', 'OptionalInfo', 'OptionalArgumentFor', 'OptionalArgumentAgainst', 'ID_CandidateQuestionType', 'QuestionListData', 'IsTopicQuestionType', 'RelatedQuestionId'])

In [16]:
#Jamen jesus DR.
# Fake "lokale" spørgsmål med unikt ID. Hara Krisna forbyde at nogen nogensinde skal bruge disse data til noget#.
#... ok......lad os unfake dem
sprgs = {}
n_sprg = [] # hvor mange spørgsmål er der per kandidat
for kandidat in drdata:
    n_sprg.append(len(kandidat['questions']))
    for sprg in kandidat['questions']:
        sprgs[sprg['Id']] = sprg['Question']

In [17]:
print(np.min(n_sprg), np.mean(n_sprg), np.median(n_sprg), np.max(n_sprg))
# hmm..de fleste har 18 spørgsmål men fra 16 til 20...

16 18.089688834655277 18.0 20


In [18]:
len(sprgs.keys())
# der er 317 spørgsmål men jeg tror de bare udskifter kommune. Lad os kigge på det

317

In [19]:
json.dump(sprgs, open('sprgs.json', 'wt'), indent=4, ensure_ascii=False)

In [20]:
# nope...de er med lignende tema men formuleret forskelligt...øj hvor besværligt.
# nå...jeg laver manuelt et map til nogle spørgsmål jeg finder på
temaer = {
    "DR0": "Flere penge til uddannelse, skoler og børnehaver",
    "DR1": "Mere privatisering",
    "DR2": "Tage hensyn og bruge flere penge på sociale mindretal",
    "DR3": "Tage hensyn og bruge flere penge på religiøse mindretal",
    "DR4": "Bedre forhold for privat trafik ",
    "DR5": "Der skal generelt bruges flere penge på velfærd",
    "DR6": "Erhverslivet skal hjælpes",
    "DR7": "Flere grønne tiltag",
    "DR8": "Mere brugerbetaling",
    "DR9": "Mere fokus på integration",
    "DR10": "Flere boliger og lejeboliger",
    "DR11": "Flere penge på idræt og kultur",
    "DR12": "Flere penge til små samfund/landsbyer/skoler",
    "DR13": "Mere offentligt indblik i kommunens aktiviteter",
    "DR14": "Kommunen skal gøre mere for at sikre borgerne mod klimaforandringerne",
    "DR15": "Flere penge til de ældre",
    "DR16": "Der skal satses mere på turisme",
    "DR17": "Bedre offentlig transport",
    "DR18": "Prioriter natur over erhverv",
    "DR19": "Mere politi og kontrol med at regler overholdes",
}

# her er de AI generede associationer mellem spørgsmål og temaer
sprg_map = pd.read_csv('TakeTheDR/sprgs_map.tsv', sep='\t')
sprg_map

,id,sprg,gruppe,retning
0,650,Flere opgaver i den offentlige sektor skal fre...,1,positiv
1,651,Offentlige institutioner tager for mange hensy...,3,negativ
2,652,Investering i vejnettet haster mere end invest...,4,positiv
3,654,Det er muligt at spare penge i den offentlige ...,5,negativ
4,656,Kommuneskatten i Randers Kommune skal sættes o...,5,positiv
...,...,...,...,...
312,998,Haderslev Kommune skal investere i infrastrukt...,-1,positiv
313,999,Guldborgsund Kommune skal være bedre til at gi...,2,positiv
314,1000,Kommunen skal sikre sommerhusområder mod overs...,14,positiv
315,1001,"Byrådet skal modarbejde, at der bliver sat elg...",-1,positiv


In [21]:
sprg_map['gruppe'].value_counts()

gruppe
 7     44
 10    37
 12    35
 0     33
-1     30
 4     24
 11    20
 18    12
 17    12
 15    12
 16    11
 14    11
 19     7
 2      5
 6      5
 3      5
 13     4
 5      3
 1      3
 9      2
 8      2
Name: count, dtype: int64

In [22]:
bogstaver = json.load(open('various.json'))['bogstav_leg']

In [23]:
# strukturen var som sagt
# 'candidate': {'ID', 'Firstname', 'LastName', 'Picture', 'CroppedPicture', 'Photographer', 'Gender', 'Birthdate', 'UrlKey', 'PersonUrlKey', 'CurrentParty', 'CurrentPartyCode', 'City', 'Zipcode', 'Facebook', 'Homepage', 'Instagram', 'Twitter', 'Profession', 'Education', 'LeisureInterest', 'PoliticsInterest', 'LinedUpForParty', 'LinedUpForPartyCode', 'Keytopics', 'LineUps', 'OnOtherElections']}
# 'candidateAnswers': [{'QuestionID': 650, 'Answer': 1, 'AnswerList': None,  'Info': '', 'IsImportant': 1, 'ID_CandidateQuestionType': 1, 'IsTopicQuestionType': False}]
# 'questions': [dict_keys(['Id', 'Title', 'Question', 'Info', 'ArgumentFor', 'ArgumentAgainst', 'OptionalTitle', 'OptionalQuestion', 'OptionalInfo', 'OptionalArgumentFor', 'OptionalArgumentAgainst', 'ID_CandidateQuestionType', 'QuestionListData', 'IsTopicQuestionType', 'RelatedQuestionId'])

dr_data = []
for dob in drdata:
    try:
        struk = {}
        struk['navn'] = dob['candidate']['Firstname'] + ' ' + dob['candidate']['LastName']
        struk['parti'] = dob['candidate']['LinedUpForParty']
        struk['parti_bogstav'] = dob['candidate']['LinedUpForPartyCode']
        struk['kreds'] = dob['candidate']['LineUps'][0]['lineUpName']

        persvar = pd.DataFrame(dob['candidateAnswers'])[['QuestionID', 'Answer']]
        persvar['nykat'] = persvar['QuestionID'].map(sprg_map.set_index('id')['gruppe'])
        persvar['retning'] = persvar['QuestionID'].map(sprg_map.set_index('id')['retning'])
        # inverter svar så 1=positiv, 5=negativ hvis sentiment er negativ
        persvar['Answer'] = persvar.apply(lambda row: 6 - row['Answer'] if row['retning']=='negativ' else row['Answer'], axis=1)
        persvar['Answer'] -= 1
        persvar['Answer'] *= 0.25
        persvar = persvar.pivot_table(index='nykat', values='Answer', aggfunc='mean')['Answer']
        persvar = pd.concat([persvar, pd.Series(index = set([x for x in range(20)]) - set(persvar.index))]).sort_index()
        persvar.index = [f"DR{int(x)}" for x in persvar.index]
        struk = struk | persvar.to_dict()
        dr_data.append(struk)
    except Exception as e:
        print(f"Fejl for kandidat {dob['candidate']['Firstname']} {dob['candidate']['LastName']}: {e}")
dr_df = pd.DataFrame(dr_data)
#dr_df = dr_df.replace(-0.25, np.nan)

Fejl for kandidat Hanne Munch Kristiansen: "None of [Index(['QuestionID', 'Answer'], dtype='object')] are in the [columns]"
Fejl for kandidat Steen Bjerregaard Jørgensen: "None of [Index(['QuestionID', 'Answer'], dtype='object')] are in the [columns]"
Fejl for kandidat Lillian Præstbro: "None of [Index(['QuestionID', 'Answer'], dtype='object')] are in the [columns]"
Fejl for kandidat Karin Müller: "None of [Index(['QuestionID', 'Answer'], dtype='object')] are in the [columns]"
Fejl for kandidat Christian Andresen: "None of [Index(['QuestionID', 'Answer'], dtype='object')] are in the [columns]"
Fejl for kandidat Brian Persson: "None of [Index(['QuestionID', 'Answer'], dtype='object')] are in the [columns]"
Fejl for kandidat Finn Hansen: "None of [Index(['QuestionID', 'Answer'], dtype='object')] are in the [columns]"
Fejl for kandidat Majbrit Franklin Kock: "None of [Index(['QuestionID', 'Answer'], dtype='object')] are in the [columns]"
Fejl for kandidat Tove Stensbjerg: "None of [Index(

In [24]:
dr_df.drop("DR-1", axis=1, inplace=True)
dr_df

,navn,parti,parti_bogstav,kreds,DR0,DR1,DR2,DR3,DR4,DR5,...,DR10,DR11,DR12,DR13,DR14,DR15,DR16,DR17,DR18,DR19
0,Mark Søgaard,Socialdemokratiet,A,Assens Kommune,1.000,0.00,NaN,0.25,0.625,0.625,...,0.750000,NaN,0.75,NaN,0.750,NaN,NaN,0.875,NaN,NaN
1,Ivan Damsted,Socialdemokratiet,A,Assens Kommune,0.250,0.00,NaN,0.25,0.625,1.000,...,0.750000,NaN,1.00,NaN,0.750,NaN,NaN,1.000,NaN,NaN
2,Kenneth Ettrup Hansen,Liberal Alliance,I,Vejle Kommune,0.000,0.75,NaN,0.25,0.750,0.125,...,0.875000,0.750,0.25,NaN,0.625,NaN,NaN,NaN,NaN,NaN
3,Kitty Gamkinn,NÆRdemokraterne,N,Varde Kommune,0.750,0.75,NaN,0.75,0.250,0.250,...,0.750000,0.250,NaN,NaN,0.250,NaN,0.50,0.750,0.75,NaN
4,Lars Storgaard,Det Konservative Folkeparti,C,Favrskov Kommune,0.250,0.75,NaN,0.75,0.750,0.375,...,1.000000,1.000,NaN,NaN,0.250,NaN,NaN,0.250,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5877,Hanne Jeppesen,Det Konservative Folkeparti,C,Skive Kommune,0.250,0.75,NaN,1.00,1.000,0.000,...,1.000000,0.250,0.25,NaN,0.250,1.0,1.00,0.250,NaN,NaN
5878,Peter Baggerman,Socialdemokratiet,A,Silkeborg Kommune,1.000,0.00,NaN,0.75,0.000,0.500,...,0.333333,0.875,1.00,NaN,0.500,NaN,NaN,NaN,NaN,NaN
5879,Jan Kjærsgaard,Socialdemokratiet,A,Odder Kommune,0.750,0.25,NaN,0.75,0.000,0.875,...,1.000000,NaN,NaN,NaN,0.000,NaN,1.00,1.000,NaN,NaN
5880,Egon Henriksen,Socialdemokratiet,A,Ringkøbing-Skjern Kommune,0.250,0.25,NaN,0.25,0.000,0.750,...,0.750000,0.750,0.00,NaN,0.750,NaN,0.25,1.000,NaN,NaN


In [25]:
len(drdata)-len(dr_df)
# hmm 2313 der ikke har svaret? Det er alligevel ret mange...hmmm

2313

In [26]:
# vi fjerner spørgsmål med mere end 15% manglende svar
fjern_dem = dr_df.columns[dr_df.isna().mean()>0.15]
pd.Series(temaer)[[int(x.split("DR")[1]) for x in fjern_dem]]
#dr_df.isna().mean()

/tmp/ipykernel_6772/290119942.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pd.Series(temaer)[[int(x.split("DR")[1]) for x in fjern_dem]]


DR2     Tage hensyn og bruge flere penge på sociale mi...
DR11                       Flere penge på idræt og kultur
DR12         Flere penge til små samfund/landsbyer/skoler
DR13      Mere offentligt indblik i kommunens aktiviteter
DR15                             Flere penge til de ældre
DR16                      Der skal satses mere på turisme
DR17                            Bedre offentlig transport
DR18                         Prioriter natur over erhverv
DR19      Mere politi og kontrol med at regler overholdes
dtype: object

In [27]:
#hmm 9 temaer fjernet. Der er alligevel ret mange...
dr_df.drop(columns=fjern_dem, inplace=True)


In [28]:
print(len(q), len(dr_df))
# ok ca lige mange har svaret på tv2 som på dr. Så er det måske alligevel ikke helt forkert
# Lad os prøve at merge så

7218 5882


In [29]:
a = set(q.parti.str.lower().unique())
b = set(dr_df.parti.str.lower().unique())
with open('parti_map.txt', 'wt') as f:
    for parti in a.union(b):
        f.write(parti + "\n")

In [30]:
# hm ok vi skal vist lige standardisere partinavne lidt først
parti_map = {"adrian-silviu suiugan": "adrian-silviu suiugan", 	"albertslund-listen": "albertslund-listen", 	"alliancen for børnene, ungdommen og fremtiden": "alliancen for børnene, ungdommen og fremtiden", 	"alliancen fr børnene, ungdommen og fremtiden": "alliancen for børnene, ungdommen og fremtiden", 	"alternativet": "alternativet",  	"anne grønlund": "anne grønlund", 	"befri ballerup": "befri ballerup", "billund international": "billund international", "bitten vivi jensen": "bitten vivi jensen", "borger-initiativet": "borger-initiativet", "borgerlig fornuft": "borgerlig fornuft", "borgerlisten": "borgerlisten", "borgerlisten hørsholm": "borgerlisten hørsholm", "borgernes gribskov": "borgernes gribskov", "borgernes liste 7000": "borgernes liste 7000", "borgernes lokal liste": "borgernes lokal liste", "borgernes lokalliste": "borgernes lokalliste", "borgernes lokalliste køge": "borgernes lokalliste køge", "borgertinget": "borgertinget", "bornholm 360": "bornholm 360", "bylisten": "bylisten", "byogland": "byogland", "corona camping listen": "corona camping listen", "danmarksdemokraterne": "danmarksdemokraterne", "danmarksdemokraterne ‒ inger støjberg": "danmarksdemokraterne", "dansk folkeparti": "dansk folkeparti", "de kreative": "de kreative", "deltagerlisten": "deltagerlisten", "demokraterne - hele danmark skal leve": "demokraterne - hele danmark skal leve", "den lille mands parti": "den lille mands parti", "den lunefulde liste": "den lunefulde liste", "den sociale fællesliste": "den sociale fællesliste", "den sociale fællesliste - rebild": "den sociale fællesliste", "det konservative folkeparti": "det konservative folkeparti", "det nære parti": "det nære parti", "det starter i odense": "det starter i odense", "din stemme": "din stemme-dit parti", "din stemme-dit parti": "din stemme-dit parti", "dit guldborgsund": "dit guldborgsund", "dkp - danmarks kommunistiske parti": "dkp - danmarks kommunistiske parti", "dkp - kommunisterne": "dkp - danmarks kommunistiske parti", "enhedslisten": "enhedslisten", "enhedslisten – de rød-grønne": "enhedslisten", "esbjerg 2.0": "esbjerg 2.0", "et bedre odense": "et bedre odense", "et samlet egedal": "et samlet egedal", "fair forvaltning": "fair forvaltning", "fanø borgerliste": "fanø borgerliste", "folkebevægelsen danmark": "folkebevægelsen danmark", "folkelisten for fred og velfærd": "folkelisten for fred og velfærd", "folkestyret.dk": "folkestyret.dk", "fredensborg socialkonservative": "fredensborg socialkonservative", "frie grønne": "frie grønne", "frie stemmer - jens peter hansen": "frie stemmer - jens peter hansen", "frit borgerligt fællesskab": "frit borgerligt fællesskab", "fælleslisten": "fælleslisten", "fælleslisten bornholm": "fælleslisten bornholm", "grøn kultur": "grøn kultur", "grøn velfærd": "grøn velfærd", "guldborgsundlisten": "guldborgsundlisten", "hampepartiet": "hampepartiet", "haslevfaxe-listen": "haslevfaxe-listen", "havdruplisten": "havdruplisten", "hvad kan gå galt med theis bach nielsen": "hvad kan gå galt med theis bach nielsen", "hverdagens stemme": "hverdagens stemme", "hvidovrelisten": "hvidovrelisten", "ib lunde's liste": "ib lunde's liste", "ildsjælene": "ildsjælene", "ishøjdemokraterne": "ishøjdemokraterne", "ishøjlisten": "ishøjlisten", "kalundborg listen": "kalundborg listen", "kludder og borgernes liste": "kludder og borgernes liste", "kommunistisk parti": "dkp - danmarks kommunistiske parti", "kristendemokraterne": "kristendemokraterne", "land- & bylisten": "land- og bylisten", "land- og bylisten": "land- og bylisten", "liberal alliance": "liberal alliance", "liste p": "liste p", "listen anne ehrenreich": "listen anne ehrenreich", "lokaldemokraterne": "lokaldemokraterne", "lokalisten fmk": "lokallisten fmk", "lokalliste l": "lokalliste l", "lokallisten": "lokallisten", "lokallisten faaborg-midtfyn": "lokallisten faaborg-midtfyn", "lokallisten fmk": "lokallisten fmk", "lokallisten gladsaxe": "lokallisten gladsaxe", "lokallisten i rudersdal": "lokallisten i rudersdal", "lokallisten jammerbugt": "lokallisten jammerbugt", "lokallisten langeland": "lokallisten langeland", "lokallisten levende landsbyer": "lokallisten levende landsbyer", "lokallisten levende landsbyer lejre": "lokallisten levende landsbyer", "lokallisten lolland": "lokallisten lolland", "lokallisten mennesker først": "lokallisten mennesker først", "lokallisten ny egedal": "lokallisten ny egedal", "lokallisten nyt odense": "lokallisten nyt odense", "lokallisten roskilde": "lokallisten roskilde", "louise isager": "louise isager", "læsø borgerliste": "læsø borgerliste", "løsgænger": "løsgænger", "miljølisten horsens": "miljølisten horsens", "moderaterne": "moderaterne", "natur- og velfærdslisten": "natur- og velfærdslisten", "nej til privatisering af kommunale veje": "nej til privatisering af kommunale veje", "ny holbæk": "ny holbæk", "nye borgerlige": "nye borgerlige", "nyt kalundborg": "nyt kalundborg", "nyt odsherred": "nyt odsherred", "nyt sjælland": "nyt sjælland", "nytgribskov": "nytgribskov", "nærdemokraterne": "nærdemokraterne", "nærdemokratiet": "nærdemokratiet", "omsorgsgruppen": "omsorgsgruppen", "psykiatri-listen": "psykiatri-listen", "radikal venstre": "radikale venstre", "radikale venstre": "radikale venstre", "rationelle": "rationelle", "rebild listen": "rebild listen", "sf - socialistisk folkeparti": "socialistisk folkeparti", "skamløse demokrater": "skamløse demokrater", "skive-listen": "skive-listen", "slesvigsk parti": "slesvigsk parti", "smilla og borgernes liste": "smilla og borgernes liste", "socialdemokratiet": "socialdemokratiet", "socialistisk folkeparti": "socialistisk folkeparti", "socialkonservative": "socialkonservative", "solidarisk ansvar": "solidarisk ansvar", "solidarisk integritet": "solidarisk integritet", "stabilt demokrati": "stabilt demokrati", "sydamagerlisten": "sydamagerlisten", "sydhavslisten": "sydhavslisten", "søren astrup": "søren astrup", "thy borgerlige": "thy borgerlige", "tværpolitisk anstændighed": "tværpolitisk anstændighed", "tværpolitisk forening": "tværpolitisk forening", "tårnby listen": "tårnby listen", "tønder listen": "tønder listen", "velfærdsdemokraterne": "velfærdsdemokraterne", "velfærdslisten": "velfærdslisten", "velfærdslisten syddjurs": "velfærdslisten", "vendelbo-listen": "vendelbo-listen", "venstre": "venstre", "venstre, danmarks liberale parti": "venstre", "verdensdemokraterne": "verdensdemokraterne", "vestlolland": "vestlolland", "vi lokale demokrater": "vi lokale demokrater", "vibeke schou - personlig frihed": "vibeke schou - personlig frihed", "vordingborg borgerliste": "vordingborg borgerliste", "vores lokalliste": "vores lokalliste", "voressolrød": "voressolrød", "ærø i centrum": "ærø i centrum", "østbroen": "østbroen"}
dr_df['parti'] = dr_df['parti'].str.lower().map(parti_map)
q['parti'] = q['parti'].str.lower().map(parti_map)
# ikke så mange af de rigtige sjove navne...måske deltagerne de ikke i online test?

In [31]:
# ensartning af kredsnavne
a = set(q.kreds.str.lower().unique())
b = set(dr_df.kreds.str.lower().unique())
with open('kreds_map.txt', 'wt') as f:
    for kreds in a.union(b):
        f.write(kreds + "\n")

In [32]:
kreds_map = {"aabenraa": "aabenraa",
"aabenraa kommune": "aabenraa",
"aalborg": "aalborg",
"aalborg kommune": "aalborg",
"aarhus": "aarhus",
"aarhus kommune": "aarhus",
"albertslund": "albertslund",
"albertslund kommune": "albertslund",
"allerød": "allerød",
"allerød kommune": "allerød",
"assens": "assens",
"assens kommune": "assens",
"ballerup": "ballerup",
"ballerup kommune": "ballerup",
"billund": "billund",
"billund kommune": "billund",
"bornholm": "bornholm",
"bornholms regionskommune": "bornholm",
"brøndby": "brøndby",
"brøndby kommune": "brøndby",
"brønderslev": "brønderslev",
"brønderslev kommune": "brønderslev",
"dragør": "dragør",
"dragør kommune": "dragør",
"egedal": "egedal",
"egedal kommune": "egedal",
"esbjerg": "esbjerg",
"esbjerg kommune": "esbjerg",
"faaborg-midtfyn": "faaborg-midtfyn",
"faaborg-midtfyn kommune": "faaborg-midtfyn",
"fanø": "fanø",
"fanø kommune": "fanø",
"favrskov": "favrskov",
"favrskov kommune": "favrskov",
"faxe": "faxe",
"faxe kommune": "faxe",
"fredensborg": "fredensborg",
"fredensborg kommune": "fredensborg",
"fredericia": "fredericia",
"fredericia kommune": "fredericia",
"frederiksberg": "frederiksberg",
"frederiksberg kommune": "frederiksberg",
"frederikshavn": "frederikshavn",
"frederikshavn kommune": "frederikshavn",
"frederikssund": "frederikssund",
"frederikssund kommune": "frederikssund",
"furesø": "furesø",
"furesø kommune": "furesø",
"gentofte": "gentofte",
"gentofte kommune": "gentofte",
"gladsaxe": "gladsaxe",
"gladsaxe kommune": "gladsaxe",
"glostrup": "glostrup",
"glostrup kommune": "glostrup",
"greve": "greve",
"greve kommune": "greve",
"gribskov": "gribskov",
"gribskov kommune": "gribskov",
"guldborgsund": "guldborgsund",
"guldborgsund kommune": "guldborgsund",
"haderslev": "haderslev",
"haderslev kommune": "haderslev",
"halsnæs": "halsnæs",
"halsnæs kommune": "halsnæs",
"hedensted": "hedensted",
"hedensted kommune": "hedensted",
"helsingør": "helsingør",
"helsingør kommune": "helsingør",
"herlev": "herlev",
"herlev kommune": "herlev",
"herning": "herning",
"herning kommune": "herning",
"hillerød": "hillerød",
"hillerød kommune": "hillerød",
"hjørring": "hjørring",
"hjørring kommune": "hjørring",
"holbæk": "holbæk",
"holbæk kommune": "holbæk",
"holstebro": "holstebro",
"holstebro kommune": "holstebro",
"horsens": "horsens",
"horsens kommune": "horsens",
"hvidovre": "hvidovre",
"hvidovre kommune": "hvidovre",
"høje-taastrup": "høje-taastrup",
"høje-taastrup kommune": "høje-taastrup",
"hørsholm": "hørsholm",
"hørsholm kommune": "hørsholm",
"ikast-brande": "ikast-brande",
"ikast-brande kommune": "ikast-brande",
"ishøj": "ishøj",
"ishøj kommune": "ishøj",
"jammerbugt": "jammerbugt",
"jammerbugt kommune": "jammerbugt",
"kalundborg": "kalundborg",
"kalundborg kommune": "kalundborg",
"kerteminde": "kerteminde",
"kerteminde kommune": "kerteminde",
"kolding": "kolding",
"kolding kommune": "kolding",
"københavn": "københavn",
"københavns kommune": "københavn",
"køge": "køge",
"køge kommune": "køge",
"langeland": "langeland",
"langeland kommune": "langeland",
"lejre": "lejre",
"lejre kommune": "lejre",
"lemvig": "lemvig",
"lemvig kommune": "lemvig",
"lolland": "lolland",
"lolland kommune": "lolland",
"lyngby-taarbæk": "lyngby-taarbæk",
"lyngby-taarbæk kommune": "lyngby-taarbæk",
"læsø": "læsø",
"læsø kommune": "læsø",
"mariagerfjord": "mariagerfjord",
"mariagerfjord kommune": "mariagerfjord",
"middelfart": "middelfart",
"middelfart kommune": "middelfart",
"morsø": "morsø",
"morsø kommune": "morsø",
"norddjurs": "norddjurs",
"norddjurs kommune": "norddjurs",
"nordfyns": "nordfyns",
"nordfyns kommune": "nordfyns",
"nyborg": "nyborg",
"nyborg kommune": "nyborg",
"næstved": "næstved",
"næstved kommune": "næstved",
"odder": "odder",
"odder kommune": "odder",
"odense": "odense",
"odense kommune": "odense",
"odsherred": "odsherred",
"odsherred kommune": "odsherred",
"randers": "randers",
"randers kommune": "randers",
"rebild": "rebild",
"rebild kommune": "rebild",
"ringkøbing-skjern": "ringkøbing-skjern",
"ringkøbing-skjern kommune": "ringkøbing-skjern",
"ringsted": "ringsted",
"ringsted kommune": "ringsted",
"roskilde": "roskilde",
"roskilde kommune": "roskilde",
"rudersdal": "rudersdal",
"rudersdal kommune": "rudersdal",
"rødovre": "rødovre",
"rødovre kommune": "rødovre",
"samsø": "samsø",
"samsø kommune": "samsø",
"silkeborg": "silkeborg",
"silkeborg kommune": "silkeborg",
"skanderborg": "skanderborg",
"skanderborg kommune": "skanderborg",
"skive": "skive",
"skive kommune": "skive",
"slagelse": "slagelse",
"slagelse kommune": "slagelse",
"solrød": "solrød",
"solrød kommune": "solrød",
"sorø": "sorø",
"sorø kommune": "sorø",
"stevns": "stevns",
"stevns kommune": "stevns",
"struer": "struer",
"struer kommune": "struer",
"svendborg": "svendborg",
"svendborg kommune": "svendborg",
"syddjurs": "syddjurs",
"syddjurs kommune": "syddjurs",
"sønderborg": "sønderborg",
"sønderborg kommune": "sønderborg",
"thisted": "thisted",
"thisted kommune": "thisted",
"tårnby": "tårnby",
"tårnby kommune": "tårnby",
"tønder": "tønder",
"tønder kommune": "tønder",
"vallensbæk": "vallensbæk",
"vallensbæk kommune": "vallensbæk",
"varde": "varde",
"varde kommune": "varde",
"vejen": "vejen",
"vejen kommune": "vejen",
"vejle": "vejle",
"vejle kommune": "vejle",
"vesthimmerlands": "vesthimmerlands",
"vesthimmerlands kommune": "vesthimmerlands",
"viborg": "viborg",
"viborg kommune": "viborg",
"vordingborg": "vordingborg",
"vordingborg kommune": "vordingborg",
"ærø": "ærø",
"ærø kommune": "ærø",
}

In [33]:
q['kreds'] = q.kreds.str.lower().map(kreds_map)
dr_df['kreds'] = dr_df.kreds.str.lower().map(kreds_map)

In [34]:
q[['navn', 'parti', 'kreds']].value_counts()

navn                      parti                    kreds      
Aage Rais                 socialistisk folkeparti  aarhus         1
Aase Bagger               socialdemokratiet        ishøj          1
Aase Pedersen             socialdemokratiet        aarhus         1
Abaz Sabani               bylisten                 glostrup       1
Abdinoor Adam Hassan      moderaterne              odense         1
                                                                 ..
Özcan Kizilkaya           socialdemokratiet        ballerup       1
Özcan Tecer               radikale venstre         køge           1
Özkan Kocak               moderaterne              brøndby        1
Øjvind Vilsholm           enhedslisten             furesø         1
Øyvind B. Fabricius Holm  venstre                  albertslund    1
Name: count, Length: 7177, dtype: int64

In [35]:
dr_df[['navn', 'parti', 'kreds']].value_counts()

navn                  parti                    kreds    
Aase Pedersen         socialdemokratiet        aarhus       1
Abaz Sabani           bylisten                 glostrup     1
Abdinoor Adam Hassan  moderaterne              odense       1
Abdul Rehman Khan     moderaterne              ishøj        1
Absalon Billehøj      enhedslisten             københavn    1
                                                           ..
Åse Mandøe            socialistisk folkeparti  holstebro    1
Özcan Kizilkaya       socialdemokratiet        ballerup     1
Özcan Tecer           radikale venstre         køge         1
Özkan Kocak           moderaterne              brøndby      1
Øjvind Vilsholm       enhedslisten             furesø       1
Name: count, Length: 5867, dtype: int64

In [36]:
# godt så skal vi lave en samlet dataframe
# vi vil kun beholde de rows hvor vi har data fra både dr og tv2
# så vi gennemgår bare DRs datafrane også ser vi om vi kan finde et match i tv2
samlet_df = []
dr_sprg_columns = dr_df.columns[dr_df.columns.str.contains("DR")]
for i, row in dr_df.iterrows():
    # kig i tv2 data efter dr folkene.
    nemt = q[(q.navn == row.navn) & (q.kreds == row.kreds) & (q.parti == row.parti)]
    if len(nemt) == 1:
        # perfekt match
        # Tilføj tv2 data til row
        samlet_df.append(pd.concat([row[dr_sprg_columns], nemt.iloc[0]]))
    elif len(nemt) == 0:
        # ikke perfekt match
        # kan skyldes at personen ikke har svaret TV2 eller at navnet er skrevet forskelligt
        måske = q[(q.navn.str.startswith(row.navn.split(" ")[0])) & (q.navn.str.endswith(row.navn.split(" ")[-1])) & (q.parti == row.parti) & (q.kreds == row.kreds)]
        if len(måske) == 1:
            # Så er det nok bare et manglende mellemnavn
            samlet_df.append(pd.concat([row[dr_sprg_columns], måske.iloc[0]]))
        elif len(måske) == 0:
            # så bliver det svært. Nok bare nogen der ikke har svaret på begge. (eller den der ene der kalder sig "Fornavn Mellemnavn" i DR og "Fornavn Mellemnavn Efternavn" i TV2 men det gider jeg altså ikke)
            pass
        else:
            # hmm flere muligheder?
            print("flere mulige", row, måske)
            break

    else:
        print("noget helt galg", nemt)
samlet_df = pd.DataFrame(samlet_df)

In [37]:
len(samlet_df)

5266

In [38]:
# 5266 kandidater er med i min test

In [39]:
# udfyld manglende svar med partiets gennemsnit
dk_spg = list(q.columns[q.columns.str.startswith("tv2")]) + list(dr_sprg_columns)
parti_linjen = round(samlet_df.groupby('parti')[dk_spg].mean()*4)/4  # gennemsnitssvaret per parti, rundet til 1-5 (0-4) svarmulighederne

def fald_ind(x):
    x.loc[x[x.isna()].index] = parti_linjen.loc[x.parti, x[x.isna()].index]
    return x
    
samlet_df_final = samlet_df.apply(fald_ind, axis=1)



In [40]:
# hvis der stadig er nogen der mangler (f.eks. fordi der var 0 i partiet der havde svaret), så giver vi dem det globale svar
genmenet_svar = samlet_df_final[dk_spg].mean()
def alm_svar(x):
    x.loc[x[x.isna()].index] = genmenet_svar.loc[x[x.isna()].index]
    return x

samlet_df_final_final = samlet_df_final.apply(alm_svar, axis=1)

In [41]:
samlet_df_final_final.to_feather("2025_KV_Lasse_data.feather")